In [56]:
%pip install matplotlib
%pip install wordcloud
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [57]:
#importing necessary libraries
import numpy
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#importing wordcloud for text visualisation
from wordcloud import WordCloud

#importing NLTK for natural language processing
import nltk
from nltk.corpus import stopwords # for stopwords

#downloading NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [58]:
#Read csv
df = pd.read_csv('spam.csv')

In [59]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [60]:
df.drop(columns = ['Unnamed: 2','Unnamed: 3','Unnamed: 4'], inplace=True)

In [61]:
#rename the columns
df.rename(columns = {'v1':'target','v2':'text'},inplace = True)

In [62]:
df.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Data Preprocessing

In [63]:
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [64]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['target'] = encoder.fit_transform(df['target'])

In [65]:
df.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [66]:
#check duplicate values
df.duplicated().sum()

np.int64(403)

In [67]:
len(df)

5572

In [68]:
#remove duplicate
df = df.drop_duplicates(keep = 'first')

Feature Engineering

In [69]:
#importing porter stemmer for text stemming

from nltk.stem.porter  import PorterStemmer

#importing the string module for handling special characters
import string

In [70]:
ps = PorterStemmer()

In [71]:
#lowercase transformation and text preprocessing function
def transform_text(text):
    #transform text to lowercase
    text= text.lower()

    #tokenization using nltk
    text = nltk.word_tokenize(text)
    
    #removing special characters 
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    #removing stop words and punctuations 
    text = y[:]
    y.clear()

    #loop through the tokens and remove stopwords and punctuations
    for i in text:
        y.append(ps.stem(i))

    #stemming using porter stemmer
    text = y[:]
    y.clear()
    
    for i in text:
        y.append(ps.stem(i))
    
    # Join the processed tokens back into a single string
    return " ".join(y)

In [72]:
transform_text('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')


'go until jurong point crazi avail onli in bugi n great world la e buffet cine there got amor wat'

In [73]:
df['transformed_text'] = df['text'].apply(transform_text)
df.head()

,target,text,transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",go until jurong point crazi avail onli in bugi...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entri in 2 a wkli comp to win fa cup fina...
3,0,U dun say so early hor... U c already then say...,u dun say so earli hor u c alreadi then say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah i do think he goe to usf he live around he...


In [75]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfid = TfidfVectorizer(max_features = 500)

In [76]:
X = tfid.fit_transform(df['transformed_text']).toarray()
y = df['target'].values

Train Test Split

In [77]:

from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state = 2)


Model Training

In [79]:
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   --- ------------------------------------ 6.8/72.0 MB 36.4 MB/s eta 0:00:02
   ------- -------------------------------- 13.6/72.0 MB 34.7 MB/s eta 0:00:02
   -------- ------------------------------- 15.5/72.0 MB 29.1 MB/s eta 0:00:02
   --------- ------------------------------ 16.5/72.0 MB 21.7 MB/s eta 0:00:03
   ---------- ----------------------------- 18.1/72.0 MB 18.2 MB/s eta 0:00:03
   ----------- ---------------------------- 20.7/72.0 MB 17.0 MB/s eta 0:00:04
   ------------- -------------------------- 24.1/72.0 MB 16.9 MB/s eta 0:00:03
   --------------- ------------------------ 28.3/72.0 MB 17.4 MB/s eta 0:00:03
   ---------------- ----------------------- 30.4/72.0 MB 16.8 MB/s eta 0:00:03
   ----------------- ---------------------- 31.5/72.0 MB 15.5 MB/s eta 0:00:03
   ------------------ --------------------- 32.8/72.0 MB 14.7 M

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [81]:
svc = SVC(kernel= "sigmoid", gamma  = 1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth = 5)
lrc = LogisticRegression(solver = 'liblinear', penalty = 'l1')
rfc = RandomForestClassifier(n_estimators = 50, random_state = 2 )
abc = AdaBoostClassifier(n_estimators = 50, random_state = 2)
bc = BaggingClassifier(n_estimators = 50, random_state = 2)
etc = ExtraTreesClassifier(n_estimators = 50, random_state = 2)
gbdt = GradientBoostingClassifier(n_estimators = 50, random_state = 2)    
xgb  = XGBClassifier(n_estimators = 50, random_state = 2)

In [82]:
clfs = {
    'SVC': svc,
    'KNN': knc,
    'NB': mnb,
    'DT': dtc,
    'LR': lrc,
    'RF': rfc,
    'Adaboost': abc,
    'Bgc': bc,
    'ETC': etc,
    'GBDT': gbdt,
    'xgb': xgb
    
}


Model Evaluation

In [83]:
from sklearn.metrics import accuracy_score, precision_score
def train_classifier(clfs, X_train, y_train, X_test, y_test):
    clfs.fit(X_train,y_train)
    y_pred = clfs.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    return accuracy , precision

In [84]:
accuracy_scores = []
precision_scores = []
for name , clfs in clfs.items():
    current_accuracy, current_precision = train_classifier(clfs, X_train, y_train, X_test, y_test)
    print()
    print("For: ", name)
    print("Accuracy: ", current_accuracy)
    print("Precision: ", current_precision)
    
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)


For:  SVC
Accuracy:  0.9729206963249516
Precision:  0.9661016949152542

For:  KNN
Accuracy:  0.9264990328820116
Precision:  1.0

For:  NB
Accuracy:  0.9680851063829787
Precision:  0.9646017699115044

For:  DT
Accuracy:  0.9361702127659575
Precision:  0.8272727272727273

For:  LR
Accuracy:  0.9671179883945842
Precision:  0.940677966101695

For:  RF
Accuracy:  0.9758220502901354
Precision:  0.9669421487603306

For:  Adaboost
Accuracy:  0.9448742746615088
Precision:  0.900990099009901

For:  Bgc
Accuracy:  0.9622823984526112
Precision:  0.896

For:  ETC
Accuracy:  0.9758220502901354
Precision:  0.9747899159663865

For:  GBDT
Accuracy:  0.9526112185686654
Precision:  0.9405940594059405

For:  xgb
Accuracy:  0.9835589941972921
Precision:  0.984
